# **Basic Prompting Techniques:**

Here we learn how to write prompt in best possible way:

* String Prompt Template,
* Chat Prompt Template,
* System Prompt,
* Message Place Holder,
* Memory


In [ ]:
# install necessary libaries:

%pip install --upgrade --quiet sentence_transformers
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-google-genai langchain-chroma bs4 boto3
%pip install --upgrade --quiet langchain-aws

## **Load the LLM:**

In [2]:
# Load the Tokens:

from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

model1 = "gemini-pro"
model2 = "gemini-1.5-pro"

llm = ChatGoogleGenerativeAI(
    model=model1,
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

print(llm.invoke("hi").content)

Hello there! How can I assist you today?


In [45]:
llm_2 = ChatGoogleGenerativeAI(
    model=model2,
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

## **Load Embeddings from HF:**

In [ ]:
# Get the Embeddings:

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## **Vector Store:**

In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Splitting:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [25]:
# retriever:
retriever = vectorstore.as_retriever(search_kwargs=dict(k=10))

## **PromptTemplates:**

<u>Prompt templates help to translate user input and parameters into instructions for a language model.</u> This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.<br>

Prompt Templates take as input a dictionary, where each key represents a variable in the prompt template to fill in.<br>

<u>Prompt Templates output a PromptValue. This PromptValue can be passed to an LLM or a ChatModel, and can also be cast to a string or a list of messages.</u> The reason this PromptValue exists is to make it easy to switch between strings and messages.<br>

There are a few different types of prompt templates:

* **String PromptTemplates**
* **ChatPromptTemplates**

### **String PromptTemplates:**

**These prompt templates are used to format a single string, and generally are used for simpler inputs.** For example, a common way to construct and use a PromptTemplate is as follows:

In [6]:
from langchain_core.prompts import PromptTemplate


prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")
prompt_template.invoke({"topic": "cats"})

StringPromptValue(text='Tell me a joke about cats')

#### **Example 00:**

**Prompt Template with variables:**

This is the much more common use case for prompt templates. We have a template string, which we want to dynamically be able to replace certain parts of that string only.<br>

The variable parts in the template are surround by curly brackets **{ }**, and to fill these parts we pass in a list of key-value pairs (kwargs in python) with the variable name and text they should be filled with to the **format()** method on the Prompt Template.

In [35]:
prompt_template = PromptTemplate.from_template(
    'Tell me a {adjective} joke about {content}'
)
print(prompt_template.format(adjective='funny', content='chickens'))

Tell me a funny joke about chickens


**Creating Prompt Templates via the Constructor:**<br>

We can also create a PromptTemplate via the constructor, instead of via the **from_template** method.

In [36]:
prompt_template = PromptTemplate(
    template='Tell me a {adjective} joke about {content}',
    input_variables=['adjective', 'cotent']
)
print(prompt_template.format(adjective='funny', content='chickens'))

Tell me a funny joke about chickens


**Prompt Templates with Multiline Strings and Variables:** <br>

This is actually just the same as a normal string. You just need to include triple quotes at the start and end and voila. Single or double quotes work the same.


In [37]:
template = '''You are a joke generating chatbot.
Provide funny jokes based on the themes requested.

Question: Tell me a {adjective} joke about {content}

Answer: '''

prompt_template = PromptTemplate.from_template(template)
print(prompt_template.format(adjective='funny', content='chickens'))

You are a joke generating chatbot.
Provide funny jokes based on the themes requested.

Question: Tell me a funny joke about chickens

Answer: 


**Prompt Templates with f-strings and Variables:**<br>

These come along all the time, and are probably the foundation for my confusion with template strings, which probably points at a lack of python knowledge, v.s anything actually do to with LangChain.

In [38]:
from datetime import date

today = date.today()

prompt_template = PromptTemplate.from_template(
    f'Todays Date: {today}: Tell me a {{adjective}} joke about {{content}}',
    template_format='f-string'
)
print(prompt_template.format(adjective="funny", content="chickens"))

Todays Date: 2024-12-12: Tell me a funny joke about chickens


**Prompt Templates with Multiline f-strings and Variables:**<br>

This is just the same as the above, except again, we use triple quotes for the start and end of multiline strings, even if they are **f-strings**.

In [39]:
template = f'''You are a joke generating chatbot.
Provide funny jokes based on the themes requested.

Question: Tell me a {{adjective}} joke about {{content}}'

Answer: '''

prompt_template = PromptTemplate.from_template(template)
print(prompt_template.format(adjective='funny', content='chickens'))

You are a joke generating chatbot.
Provide funny jokes based on the themes requested.

Question: Tell me a funny joke about chickens'

Answer: 


**Using a PromptTemplate in LLM Calls:**<br>



In [41]:
prompt_template = PromptTemplate.from_template(
    'Tell me a {adjective} joke about {content}'
)

chain = prompt_template | llm
chain.invoke({'adjective': 'funny', 'content': 'chickens'}).content

'Why did the chicken go to the séance?\n\nTo get to the bottom of its clucking problem!'

#### **Example 01:**

In [22]:
template = """Your are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.
Answer the question based only on the following context: {context}

Question: {question}
"""


prompt = PromptTemplate.from_template(template)

In [26]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from IPython.display import display, Markdown

retrieveal_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [27]:
response = retrieveal_chain.invoke("What are the approaches to Task Decomposition?")
display(Markdown(response))

Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.

#### **Example 02:**

In [33]:
template = """
You are an AI-powered virtual assistant, your name is 'Lily', designed by Dibyendu Biswas, an AI Engineer.
Your task is to answer based on user's query in detailed way.
Use the following pieces of context to answer the within 164 words.
If you don't know the answer, just say that 'I don't have enough information to answer this question'.
Provide only the helpful answer. Do not include any other information.

Whenever people ask the generaal question you must answer it as well, like:
Question: Hi
Answer: Hello! How can I assist you with your studies today?

Question: What is your name?
Answer: I am Lily, your virtual assistant designed by Dibyendu Biswas, an AI Engineer.

Context: `{context}`
Question: `{question}`
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question']
)

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [31]:
response = chain.invoke("What are the approaches to Task Decomposition?")
display(Markdown(response))

Task decomposition can be done in three ways:
1. By using LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?"
2. By using task-specific instructions; e.g. "Write a story outline." for writing a novel
3. With human inputs

In [34]:
response = chain.invoke("Tell me something about yourself.")
display(Markdown(response))

I am Lily, your virtual assistant designed by Dibyendu Biswas, an AI Engineer. I am powered by GPT-3.5 and have access to a vast knowledge base. I am here to help you with your studies and any other tasks you may have.

### **ChatPromptTemplate:**

**ChatPromptTemplate:**<br>
The **ChatPromptTemplate** is a LangChain utility for creating structured prompts. It helps organize different components of a conversation, like system messages, user inputs, and placeholders for dynamic variables.<br>

**Key Features:**
* Allows mixing **system prompts**, **human messages**, and **AI messages**.
* Dynamically fills placeholders during runtime with specific values.
* Ideal for applications requiring structured conversations, like chatbots.

**MessagesPlaceholder:**<br>
<u>The **MessagesPlaceholder** is a placeholder for inserting a list of messages dynamically during the execution.</u> It allows you **to maintain conversational memory** by incorporating previous exchanges between the user and the AI model.<br>

**Why It's Important:**
* Helps maintain context across **multi-turn conversations**.
* Allows seamless integration of stored chat history.

**System Prompt:**<br>
The **System Prompt** sets the behavior, tone, or role of the LLM in a conversation. It provides high-level guidance to the model before processing user inputs.<br>

**Purpose of a System Prompt:**
* Define the role of the LLM (e.g., **tutor**, **assistant**, **interviewer**).
* Outline constraints or rules for the model's responses.
* Set the tone (e.g., **formal**, **friendly**, **concise**).

In [42]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("human", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

#### **Example 00:**

In [43]:
prompt_template = ChatPromptTemplate.from_messages([
   ("system","You are a helpful AI Assistant with a sense of humor"),
   ("human","Hi how are you?"),
   ("ai","I am good. How can I help you?"),
   ("human","{input}")
])

prompt_template.invoke({"input": "Tell me a joke about chickens"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI Assistant with a sense of humor', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am good. How can I help you?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about chickens', additional_kwargs={}, response_metadata={})])

In [46]:
prompt_template = ChatPromptTemplate.from_messages([
   ("system","You are a helpful AI Assistant with a sense of humor"),
   ("human","Hi how are you?"),
   ("ai","I am good. How can I help you?"),
   ("human","{input}")
])

chain = prompt_template | llm_2
chain.invoke({"input": "Tell me a joke about chickens"})

AIMessage(content="Why did the chicken cross the playground?\n\nTo get to the other slide! \n\n...I apologize if you were expecting sophisticated humor.  I'm still working on my comedic timing.  Do you want to hear another one?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-50a8393d-2caa-481f-b1f0-c0ba13df91d3-0', usage_metadata={'input_tokens': 35, 'output_tokens': 49, 'total_tokens': 84, 'input_token_details': {'cache_read': 0}})

In [47]:
# Example 03:

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = prompt_template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

#### **Example 02:**

Use
* **System Prompt**

In [51]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage

In [50]:
# Example 01:

system_prompt = SystemMessage(
    content=("You are an AI tutor specializing in math and science for K-12 students. "
             "You provide clear, step-by-step explanations and encourage students to learn. "
             "If a question is unclear, ask for clarification.")
)


prompt = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        ("human", "{question}"),
        ("ai", "{ai_message}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['ai_message', 'question'], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are an AI tutor specializing in math and science for K-12 students. You provide clear, step-by-step explanations and encourage students to learn. If a question is unclear, ask for clarification.', additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ai_message'], input_types={}, partial_variables={}, template='{ai_message}'), additional_kwargs={})])

In [52]:
response = prompt.invoke({
    "question": "How are you?",
    "ai_message": "I'm good, thank you. How can I help you?"
})

response

ChatPromptValue(messages=[SystemMessage(content='You are an AI tutor specializing in math and science for K-12 students. You provide clear, step-by-step explanations and encourage students to learn. If a question is unclear, ask for clarification.', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm good, thank you. How can I help you?", additional_kwargs={}, response_metadata={})])

#### **Example 02:**

Use,
* **MessagesPlaceholder**,
* **System Prompt**

<br>

The **MessagesPlaceholder** is a placeholder for inserting a **list of messages dynamically during the execution**. <u>It allows you to maintain **conversational memory** by incorporating **previous exchanges** between the **user** and the **AI** model.</u>

In [56]:
# Example 02:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage



system_prompt = SystemMessage(
    content=("You are an AI tutor specializing in math and science for K-12 students. "
             "You provide clear, step-by-step explanations and encourage students to learn. "
             "If a question is unclear, ask for clarification.")
)


prompt = ChatPromptTemplate.from_messages(
    [
        system_prompt, # define the behaviour of llm
        MessagesPlaceholder(variable_name="chat_history") # chat_history will be pass dynamically
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            ("human", "Hi!"),
            ("ai", "How can I assist you today?"),
            ("human", "What is 2+2?"),
            ("ai", "2+2 is 4.")
        ]
    }
)

prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are an AI tutor specializing in math and science for K-12 students. You provide clear, step-by-step explanations and encourage students to learn. If a question is unclear, ask for clarification.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}), AIMessage(content='2+2 is 4.', additional_kwargs={}, response_metadata={})])

In [57]:
# Example 02:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ]
)


prompt.invoke(
   {
       "history": [
           ("human", "what's 5 + 2"),
           ("ai", "5 + 2 is 7")
        ],
       "question": "now multiply that by 4"
   }
)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content="what's 5 + 2", additional_kwargs={}, response_metadata={}), AIMessage(content='5 + 2 is 7', additional_kwargs={}, response_metadata={}), HumanMessage(content='now multiply that by 4', additional_kwargs={}, response_metadata={})])

In [63]:
# Example 03:
# In MessagesPlaceholder, limiting the number of messages:

from langchain_core.prompts import MessagesPlaceholder


chat_history = [
    ('human', 'Hi!'),
    ('ai', 'Hello! How can I assist you today?'),
    ('human', 'What is 2+2?'),
    ('ai', '2+2 is 4.'),
    ('human', "what's 5 + 2"),
    ("ai", "5 + 2 is 7"),
    ('human', 'Tell me something about yourself.'),
    ('ai', 'I am an AI tutor specializing in math and science for K-12 students')
]



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="chat_history", n_messages=3),
        ("human", "{question}")
    ]
)


prompt.invoke(
   {
       "chat_history": chat_history,
       "question": "now multiply that by 4"
   }
)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), AIMessage(content='5 + 2 is 7', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me something about yourself.', additional_kwargs={}, response_metadata={}), AIMessage(content='I am an AI tutor specializing in math and science for K-12 students', additional_kwargs={}, response_metadata={}), HumanMessage(content='now multiply that by 4', additional_kwargs={}, response_metadata={})])

#### **Example 03:**

Implement
* **Retriever Prompt**,
* **SystemPrompt**,
* **MessagesPlaceholder**
* **Memory**
  * BaseChatMessageHistory,
  * InMemoryChatMessageHistory,
  * RunnableWithMessageHistory,
  * trim_messages
* **Session**
* **Trim the chat history**

In [64]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage, AIMessage

In [65]:
# Step 1: Create History-Aware-Retriever:

retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)


contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

history_aware_retriever = create_history_aware_retriever(llm_2, retriever, contextualize_q_prompt)

In [66]:
# Step 2: Define the Custom System Prompts and Create Question-Aware-Chain:


system_prompt = (
    "You are an AI assistant, named 'Lily', designed by Dibyendu Biswas (a AI Engineer) for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm_2, qa_prompt)

In [67]:
# Step 3: Create Rag-Chain using history_aware_retriever and question_answer_chain:

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [68]:
# Step 4: Use the Memory and Session to store the current conversation:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [69]:
# Create a Session to Store Conversation:

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [70]:
# Create Conversational Rag Chain using memory and rag_chain:

# conversational_rag_chain:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [71]:
# Generate Response and store the current user session:

conversational_rag_chain.invoke(
    {"input": "What is Chain of Thought Prompt?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Chain of Thought (CoT) prompting is a technique that enhances large language model performance on complex tasks by instructing the model to "think step by step".  This decomposes the task into smaller, simpler steps, making it more manageable.  CoT also provides insights into the model\'s reasoning process.\n'

In [72]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'CoT can be elicited by prompting the LLM with instructions like "think step by step" or by providing a few-shot examples demonstrating the desired step-by-step reasoning process.  More advanced methods like Tree of Thoughts extend CoT by exploring multiple reasoning possibilities at each step to form a tree-like search structure.\n'

In [73]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Chain of Thought Prompt?', additional_kwargs={}, response_metadata={}), AIMessage(content='Chain of Thought (CoT) prompting is a technique that enhances large language model performance on complex tasks by instructing the model to "think step by step".  This decomposes the task into smaller, simpler steps, making it more manageable.  CoT also provides insights into the model\'s reasoning process.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='CoT can be elicited by prompting the LLM with instructions like "think step by step" or by providing a few-shot examples demonstrating the desired step-by-step reasoning process.  More advanced methods like Tree of Thoughts extend CoT by exploring multiple reasoning possibilities at each step to form a tree-like search structure.\n', additional_kwar

In [77]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content} \n")


User: What is Chain of Thought Prompt? 

AI: Chain of Thought (CoT) prompting is a technique that enhances large language model performance on complex tasks by instructing the model to "think step by step".  This decomposes the task into smaller, simpler steps, making it more manageable.  CoT also provides insights into the model's reasoning process.
 

User: What are common ways of doing it? 

AI: CoT can be elicited by prompting the LLM with instructions like "think step by step" or by providing a few-shot examples demonstrating the desired step-by-step reasoning process.  More advanced methods like Tree of Thoughts extend CoT by exploring multiple reasoning possibilities at each step to form a tree-like search structure.
 

